<a href="https://colab.research.google.com/github/ChaeYun430/MachineLearningStudy25/blob/master/scikit_learn/cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sklearn
print(sklearn.__version__)
# 지도학습 : 학습을 위한 다양한 피처와 분류 결정값인 Label 데이터로 모델을 학습한 뒤, 별도의 테스트 데이터 세트에서 미지의 레이블 예측
#            명확한 정답이 주어진 데이터를 먼저 학습한 뒤 미지의 정답 예측
# 하이퍼 파라미터 : 머신러닝 알고리즘별로 최적의 학습을 위해 직접 입력하는 파라미터들을 통칭, 성능 튜닝 가능

In [2]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import pandas as pd

# === 데이터 세트 활용 ===
iris  = load_iris()
keys = iris.keys()
print(keys)

iris_feature = iris.feature_names
iris_feature_data  = iris.data
iris_label = iris.target_names
iris_label_data =  iris.target
print('type : {} \nfeature : {} \nfeature_data  : {} \nlabel : {} \nlabel_data : {}'\
       .format(type(iris), iris_feature, iris_feature_data.tolist(), iris_label, iris_label_data.tolist()))

#  ===== Scikit-learn framework =====
#  지도학습 클래스 Estimator -> Classifier, Regressor
#  비지도학습의 fit() : 입력 데이터의 형태에 맞춰  데이터를 변환하기 위함 사전 구조를 맞추는 작업 수행

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
type : <class 'sklearn.utils._bunch.Bunch'> 
feature : ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'] 
feature_data  : [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2], [4.7, 3.2, 1.3, 0.2], [4.6, 3.1, 1.5, 0.2], [5.0, 3.6, 1.4, 0.2], [5.4, 3.9, 1.7, 0.4], [4.6, 3.4, 1.4, 0.3], [5.0, 3.4, 1.5, 0.2], [4.4, 2.9, 1.4, 0.2], [4.9, 3.1, 1.5, 0.1], [5.4, 3.7, 1.5, 0.2], [4.8, 3.4, 1.6, 0.2], [4.8, 3.0, 1.4, 0.1], [4.3, 3.0, 1.1, 0.1], [5.8, 4.0, 1.2, 0.2], [5.7, 4.4, 1.5, 0.4], [5.4, 3.9, 1.3, 0.4], [5.1, 3.5, 1.4, 0.3], [5.7, 3.8, 1.7, 0.3], [5.1, 3.8, 1.5, 0.3], [5.4, 3.4, 1.7, 0.2], [5.1, 3.7, 1.5, 0.4], [4.6, 3.6, 1.0, 0.2], [5.1, 3.3, 1.7, 0.5], [4.8, 3.4, 1.9, 0.2], [5.0, 3.0, 1.6, 0.2], [5.0, 3.4, 1.6, 0.4], [5.2, 3.5, 1.5, 0.2], [5.2, 3.4, 1.4, 0.2], [4.7, 3.2, 1.6, 0.2], [4.8, 3.1, 1.6, 0.2], [5.4, 3.4, 1.5, 0.4], [5.2, 4.1, 1.5, 0.1], [5.5, 4.2,

In [12]:
# ===== 학습/테스트 데이터 세트 분리 =====
from sklearn.model_selection import train_test_split
# test_size = 0.25
# shuffle = True
# random_state

X_train, X_test, Y_train, Y_test = train_test_split(iris_feature_data, iris_label_data, test_size= 0.2)
print(train_test_split.__module__)

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, Y_train)

pred = dt_clf.predict(X_test)

accuracy = accuracy_score(Y_test,  pred)
print(accuracy)
# feature&label data를  train&test data로 나눈다.
# 알고리즘  객체에 학습 후 test data 대입으로 target data를 관찰한다.
# 예측값과 준비한 target data를 비교하여  정확도를 측정한다.
# 데이터 세트 분리(피처 처리)-> 모델 학습 -> 예측 수행 -> 평가

sklearn.model_selection._split
0.9333333333333333


In [5]:
# 과적합(Overfitting)
# 모델이 학습 데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행할 경우에는 예측 성능이 과도하게 떨어지는 것

# 교차 검증
# 데이터 편중을 막기 위해 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행하는 것

# ===== K 폴드 교차 검증 =====
# K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
import numpy as np

kfold = KFold(n_splits=5)
cv_accuracy = []
print(iris_feature_data.shape[0])

n_iter = 0
for train_index, test_index in kfold.split(iris_feature_data):
  X_train, X_test = iris_feature_data[train_index], iris_feature_data[test_index]
  y_train, y_test = iris_label_data[train_index], iris_label_data[test_index]

  dt_clf = DecisionTreeClassifier()
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_test)
  n_iter += 1

  accuracy = np.round(accuracy_score(y_test, pred), 4)
  train_size = X_train.shape[0]
  test_size = X_train.shape[0]
  print(n_iter, accuracy, train_size, test_size, test_index)
  cv_accuracy.append(accuracy)

print(np.mean(cv_accuracy))

150
1 1.0 120 120 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
2 0.9667 120 120 [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
3 0.9 120 120 [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]
4 0.9333 120 120 [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
5 0.7333 120 120 [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
0.9066599999999999
